In [1]:
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np

In [2]:
fhand_L6_t_lapserate_day = open(r'C:\Users\Abhilash Mittal\Desktop\All_T_data\L6_t_lapserate_day_15min.txt')
julian_day = []
t_15min = []
lr_15min = []
for line in fhand_L6_t_lapserate_day:
    julian_day.append(float(line.split()[0]))
    t_15min.append(float(line.split()[1]))
    lr_15min.append(float(line.split()[2]))

Hourly_temp = []
daily_average_lr = []
for x in range(0, len(t_15min), 4):
    chunk = t_15min[x:x+4]
    Hourly_temp.append(sum(chunk)/len(chunk))
    
for x in range(0, len(lr_15min), 96):
    chunk = lr_15min[x:x+96]
    z = 24
    while z > 0:
        daily_average_lr.append(sum(chunk)/len(chunk))    
        z= z - 1
sum(daily_average_lr)/len(daily_average_lr)

0.006211835224201173




## PDD Calculation

In [3]:
day = []
T_data_file = open('C:\\Users\\Abhilash Mittal\\Desktop\\All_T_data\\ERA5LAND\\9_tiles_T\\9_tiles_T_data.txt')
for line in T_data_file:
    day.append(float(line.split()[0]))

fhand_ablation = open('F:\Abhilah_First\Ablation_data_2017_reduced.txt')

def ddf(range_days, elevation, melt):
    temp = []
    for time_step, lr, temp_at_l6 in zip(day, daily_average_lr, Hourly_temp):
        if int(time_step) in range_days:
            temp.append((lr) * (4376 - elevation) + temp_at_l6)

    # now for pdd calculation I need positive values from  x hourly_temp list
    PDD = sum([x for x in temp if x > 0])/24    # divided by 24 because delta t is 1/24

    return PDD

pdd = []

for line in fhand_ablation:
    n = int(line.split()[3]) # n is observation period
    last_day = int(line.split()[2])
    starting_day = last_day - n + 1
    range_days = list(range(starting_day, last_day + 1))
    pdd.append(ddf(range_days, int(line.split()[4]), float(line.split()[6])))
    print(ddf(range_days, int(line.split()[4]), float(line.split()[6])))


186.37502896336147
155.84897561457174
109.8951994917797
79.25254983927488
128.5934793596743
121.84508800776855
184.67282460196554
153.70081670041674
108.08791567633607
78.02909389330297
126.00858549934378
185.2103628213537
154.345264374663
108.99155758405789
78.60862565718443
127.36699352537154
100.08641908259767
132.67730341733002
191.12955114318314
185.8374907439733
155.09711999461737
188.12884548639468
109.26265015637443
78.93058774822968
127.91209834061728
158.54417769904066
184.94159371165972
154.02304053754
108.44937243942496
78.22227114793013
126.55158684251965
183.8665172728831
152.73414518904687
184.72420008880817
107.3650021501586
77.44956212942145
124.7905078009317
128.53280743539497
180.28292914362848
148.54523530644457
103.75043451927148
74.93825781926832
119.41183247810041
94.60900044186998
127.48196440762183
128.71837669536905
180.55169825332283
148.86745914356777
104.021527091588
75.19582749210456
120.08220019513878
118.74169027915745
93.95498030566377
126.6947918303934

In [4]:
db_thickness = []
melt = []
ob_day = []
melt_rate = []
fhand_ablation2 = open('F:\Abhilah_First\Ablation_data_2017_reduced.txt')

for line in fhand_ablation2:
    db_thickness.append(float(line.split()[5]))
    melt.append(float(line.split()[6]))
    ob_day.append(float(line.split()[3]))
    melt_rate.append(float(line.split()[6])/float(line.split()[3]))

# Fitting model 2.1

In [6]:
def func(X, ddf, d0):
    p, d = X
    return ddf * p / (1 + (d / d0))/10 # divided by 10 to make it in  cm, ddf is in mm/(day*C)
p = pdd # in C*day
d = db_thickness # in cm
yData = melt # in cm
initial_values = [5, 10]
popt, pcov = curve_fit(func, (p, d), yData, p0=np.asarray(initial_values))  # fitting the data in the model
print("Model2.1 : ", popt, " Initial values ", initial_values)

Model2.1 :  [5.77326438 9.62538702]  Initial values  [5, 10]


## Fitting for Model 2.2

In [12]:
def func2(X, ddf, d0):
    p, d = X
    return ddf * p / (1 + (d / d0))  # ddf in cm/(C*day^2)

yData2 = melt_rate # in cm/day
initial_values2 = [1, 10]
popt2, pcov2 = curve_fit(func2, (p, d), yData2, p0=np.asarray(initial_values2))  # fitting the data in the model
print("Model2.2 : ", popt2, " Initial values ", initial_values2)

Model2.2 :  [ 0.03022164 10.55357305]  Initial values  [1, 10]
